In [1]:
from sklearn.datasets import load_files # text로부터 숫자로 바꾸는 것 (텍스트 분석을 위해)

In [61]:
reviews_train = load_files('data-files/aclImdb/train') # 지정된 폴더(Train)의 하위 폴더가 타겟 레이블-> neg, pos

In [3]:
print(reviews_train.keys())
print(len(reviews_train['data']))
print(reviews_train['data'][0])
print(reviews_train['target_names'])
print(reviews_train['target'])

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
25000
b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."
['neg', 'pos']
[1 0 1 ... 0 0 0]


In [62]:
X_train, y_train = reviews_train['data'], reviews_train['target']

In [63]:
import numpy as np
import random

In [64]:
random.seed(0)
size = int(len(X_train) * 0.2)
# idxes = np.random.randint(0, len(X_train), size) # 20%의 데이터만 샘플링 # randint는 비복원추출만 가능, 복원 추출 불가
idxes = random.sample(range(len(X_train)), size) # 복원 추출 가능
print( len(idxes), idxes[:5], len(np.unique(idxes)) )

5000 [12623, 24836, 13781, 1326, 8484] 5000


In [65]:
# X_train2, y_train2 = [X_train[i] for i in idxes], [y_train[i] for i in idxes]
X_train2, y_train2 = np.array(X_train)[idxes], np.array(y_train)[idxes] # np.array로 만든 후 정렬
X_train2.shape, y_train2.shape

((5000,), (5000,))

In [66]:
X_train2 = np.array([x.replace(b'<br />', b' ') for x in X_train2]) # b = data를 각 1byte형식의 array로 만들어줌 # r이면 \n 안먹음
# X_train2 = map(lambda x: x.replace(b'<br />', b' '), X_train2)
# X_train2 = np.array(list(X_train3)) # map인경우 리스트, np.array를 해줘야 프린트 결과물이 나옴
X_train2[1]

b"Friz Freleng's 'Speedy Gonzalez' was the second cartoon to feature the title character after Robert McKimson's 'Cat-tails for Two'. In that cartoon, Speedy has been an ugly little creature with a big gold tooth but by his second appearance the famous design had already been adopted. Despite looking significantly more handsome, Speedy never developed into much of a character. A big hat, tremendous speed and a bad Mexican accent do not a classic character make and that's pretty much all Speedy ever had going for him. Nevertheless, the cocky little mouse proved enormously popular and went on to star in many shorts including some truly abysmal films from the studio's latter days. While these early Speedy shorts are better than those later atrocities in which he was frequently (rather oddly) paired up with Daffy Duck, they still leave much to be desired, relying on predictable gags usually based around a similar chase formula. In this self-titled episode, Speedy is recruited by some other

In [67]:
np.unique(y_train2, return_counts=True) #불균형성 확인 -> 균형적인 편

(array([0, 1]), array([2505, 2495], dtype=int64))

In [10]:
reviews_test = load_files('data-files/aclImdb/test')

In [68]:
X_test, y_test = reviews_test['data'], reviews_test['target']
size = int(len(X_test) * 0.2)
idxes = random.sample(range(len(X_test)), size)
len(np.unique(idxes))

5000

In [69]:
X_test2 = np.array([ X_test[idx] for idx in idxes])
y_test2 = np.array([ y_test[idx] for idx in idxes])

In [70]:
np.unique(y_test2, return_counts=True) # 비율 상 문제 없다

(array([0, 1]), array([2544, 2456], dtype=int64))

In [71]:
X_test2 = np.array([ x.replace(b'<br />', b' ') for x in X_test2])

In [72]:
X_test2[0]

b'Hayao Miyazaki\'s latest and eighth film for Studio Ghibili, "Gake No Ue No Ponyo" (Ponyo on the Cliff by the Sea) is a wonderfully fun and imaginative look at childhood. At a time when it seems that film animation has been dominated by Disney/Pixar\'s CGI masterpieces, it is both refreshing and comforting to know that Miyazaki is still relying on traditional hand-drawn animation to tell his charming and enchanting stories.   The story revolves around the friendship between a magical sea sprite/goldfish and the human child that she encounters during a curious outing to see the human world. The human child, Sosuke (Doi Hiroki) lives in a small house on a cliff overlooking a small port city in Southern Japan (based on Seto Island) where he lives with his young mom, Lisa (Yamaguchi Tomoko). Sosuke names the strange goldfish "Ponyo" and takes it to the daycare/nursing center that Lisa works at. Ponyo is definitely not your typical goldfish and soon begins to adapt and take on human aspec

### BOW example

In [73]:
test_words = ['The fool doth think he is wise, ', 'but the wise man knows himself to be a fool']

In [74]:
from sklearn.feature_extraction.text import CountVectorizer

In [75]:
cv = CountVectorizer() # 문장 -> 단어 사전 만들기 + 단어별 빈도수 계산하는 도구
cv.fit(test_words)

CountVectorizer()

In [76]:
cv.vocabulary_

{'the': 9,
 'fool': 3,
 'doth': 2,
 'think': 10,
 'he': 4,
 'is': 6,
 'wise': 12,
 'but': 1,
 'man': 8,
 'knows': 7,
 'himself': 5,
 'to': 11,
 'be': 0}

In [77]:
import pandas as pd

# print(cv.transform(test_words)) # 0번째 문장에 2번(doth) 하나 ..
print(cv.transform(test_words).toarray()) # 0번째 문장에 0번  x 3번,4,5번 존재
pd.DataFrame(cv.transform(test_words).toarray(), columns=sorted(cv.vocabulary_, key=lambda key: cv.vocabulary_[key])) # 빈도수 계산

[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


,be,but,doth,fool,he,himself,is,knows,man,the,think,to,wise
0,0,0,1,1,1,0,1,0,0,1,1,0,1
1,1,1,0,1,0,1,0,1,1,1,0,1,1


In [78]:
cv = CountVectorizer()
cv.fit(X_train2)
X_train3 = cv.transform(X_train2)

In [79]:
print(X_train3.toarray().shape)# 문장 5000, 단어 38326개
print(X_train3.toarray()[:5])
# print(X_train3)
# print(cv.vocabulary_) # 각각의 단어에 대해 번호를 다 붙인다
# cv.get_feature_names() # 정렬된 컬럼 이름들만 정리

(5000, 38326)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

In [81]:
lr = LogisticRegression(max_iter=10000, C=0.1)
lr.fit(X_train3, y_train2)
print(lr.score(X_train3, y_train2))
# print(lr.score(X_test3, y_test2))

0.9934


In [82]:
X_test3 = cv.transform(X_test2)

In [83]:
print(lr.score(X_test3, y_test2))

0.8612


In [84]:
print(average_precision_score(y_test2,lr.predict_proba(X_test3)[:, 1]))
print(roc_auc_score(y_test2, lr.predict_proba(X_test3)[:, 1]))

0.9190131094383963
0.9284378329031202


In [85]:
cv = CountVectorizer(min_df=5) # 5회 이상 발견된 데이터만 포함
cv.fit(X_train2)
print(len(cv.get_feature_names()))
X_train3 = cv.transform(X_train2)
X_test3 = cv.transform(X_test2)

11327


In [86]:
lr = LogisticRegression(max_iter=10000, C=0.1)
lr.fit(X_train3, y_train2)
print(lr.score(X_train3, y_train2))
print(lr.score(X_test3, y_test2)) # test data로 보니 호/불호를 맞추는 확률이 86%

0.9884
0.861


In [87]:
len(cv.get_feature_names())

11327

In [88]:
X_train2[0]

b'I saw this not too long ago, and I must say: This movie is terrible. I watch crappy movies for fun. Scarecreow is not fun. Scarecrow is stupid. You have an incredibly corny villain that enjoys screaming awful puns as he kills his victims(actually worse than the one contained in this sentence). He has his hard luck story that he uses to justify his killings. "Everyone picks on me. The only girl that thinks I\'m not trailer-trash likes one of the guys that pick on me. I want to kill everybody. Wah." OK, I\'m exaggerating. But the premise to this movie alone is enough to put it near the bottom of the list of crappy movies.  Adding to what I just said, the kid\'s mom is promiscuous, he walks in on his mother and her current boyfriend getting it on, mom\'s boyfriend tells him to leave, kid refuses, insisting that he isn\'t going to leave his own house. Boyfriend chases kid into corn field. He kills kid right in front of mom, mom screams in terror, boyfriend is like, "OMG! I didn\'t mean t

In [89]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS # 불용어, 의미를 갖지 않는 단어들, 굳이 써도 되지 않는 단어들

list(ENGLISH_STOP_WORDS)[:10]

['rather',
 'yours',
 'along',
 'for',
 'at',
 'ours',
 'move',
 'that',
 'were',
 'fifty']

In [90]:
cv = CountVectorizer(min_df=5, stop_words=ENGLISH_STOP_WORDS) # 기존 불용어 세트를
# cv = CountVectorizer(min_df=5, stop_words='english') # 영어를 불용어로
cv.fit(X_train2)
X_train3 = cv.transform(X_train2)
X_test3 = cv.transform(X_test2)

In [91]:
lr=LogisticRegression(max_iter=1000, C=0.1)
lr.fit(X_train3, y_train2)
lr.score(X_train3, y_train2), lr.score(X_test3, y_test2)


(0.9846, 0.857)

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer # 상대빈도

tv = TfidfVectorizer(min_df=5) # 단어의 상대빈도 계산 (단일 문서 내 단어빈도 / 이단어가 출연한 문서빈도)
tv.fit(X_train2)
X_train3 = tv.transform(X_train2)
X_test3 = tv.transform(X_test2)

In [94]:
lr = LogisticRegression(max_iter=1000, C=100)
lr.fit(X_train3, y_train2)
lr.score(X_train3, y_train2), lr.score(X_test3, y_test2)

(1.0, 0.8618)

### n-grams example


In [95]:
test_words = ['The fool doth think he is wise, ', 'but the wise man knows himself to be a fool']

In [96]:
from sklearn.feature_extraction.text import CountVectorizer # 존재하지 않는 특성을 가공

In [97]:
cv = CountVectorizer()
cv.fit(test_words)
data = cv.transform(test_words)
print(cv.get_feature_names())
data.toarray() # be가 0,1 ...

['be', 'but', 'doth', 'fool', 'he', 'himself', 'is', 'knows', 'man', 'the', 'think', 'to', 'wise']


array([[0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1],
       [1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]], dtype=int64)

In [98]:
cv = CountVectorizer(ngram_range=(2,2)) # default= 1,1(min,max) # 2단어씩 # (1,2) -> '한단어', '두단어', '한단어', '두단어'...
cv.fit(test_words)
data = cv.transform(test_words)
print(cv.get_feature_names())
data.toarray() # be가 0,1 ...

['be fool', 'but the', 'doth think', 'fool doth', 'he is', 'himself to', 'is wise', 'knows himself', 'man knows', 'the fool', 'the wise', 'think he', 'to be', 'wise man']


array([[0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1]], dtype=int64)

In [99]:
tv = TfidfVectorizer(min_df=5, ngram_range=(1,2)) # 적어도 5군데 이상에서 출연해야 뽑겠다 max_df = 5군데 이내(6군데 이상일 시 안뽑음)
tv.fit(X_train2)
X_train3 = tv.transform(X_train2)
X_test3 = tv.transform(X_test2)

In [100]:
lr = LogisticRegression(max_iter=10000, C=10)
lr.fit(X_train3, y_train2)
lr.score(X_train3, y_train2), lr.score(X_test3, y_test2)

(0.9996, 0.8798)

In [101]:
# !pip install spacy # 표제어 사전 관리도구 설치
# !python -m spacy download en # 영어 표제어 사전 설치

In [102]:
import nltk
import spacy

In [103]:
# 어간추출기
stemmer = nltk.stem.PorterStemmer() # 어간 추출기
en_nlp = spacy.load('en') # 영어 표제어 사전 가져오기

In [104]:
data = u"Our meeting today was worse than yesterday, I'm scared of meeting the clients tomorrow"

In [105]:
spacy_doc = en_nlp(data)
print([ token for token in spacy_doc]) # 분리된 단어 목록 출력
print([ token.lemma_ for token in spacy_doc]) # 개별 단어의 표제어 출력 -> was/were -> be로 출력 . 가장 정확도 높다
print([ token.norm_ for token in spacy_doc]) # 개별 단어의 정규화 표현 출력
print([ stemmer.stem(token.norm_) for token in spacy_doc ]) # 개별 단어의 어간 출력

[Our, meeting, today, was, worse, than, yesterday, ,, I, 'm, scared, of, meeting, the, clients, tomorrow]
['-PRON-', 'meeting', 'today', 'be', 'bad', 'than', 'yesterday', ',', '-PRON-', 'be', 'scared', 'of', 'meet', 'the', 'client', 'tomorrow']
['our', 'meeting', 'today', 'was', 'worse', 'than', 'yesterday', ',', 'i', 'am', 'scared', 'of', 'meeting', 'the', 'clients', 'tomorrow']
['our', 'meet', 'today', 'wa', 'wors', 'than', 'yesterday', ',', 'i', 'am', 'scare', 'of', 'meet', 'the', 'client', 'tomorrow']


In [108]:
import re 
from sklearn.feature_extraction.text import CountVectorizer
regexp = re.compile(r'(?u)\b\w\w+\b') # 정규 표현식 # 무조건 r을 넣어줘야 한다.

def custom_tokenizer(src):
    from spacy.tokens import Doc
    doc = Doc(en_nlp.vocab, words=regexp.findall(src))
    return [ token.lemma_ for token in doc ]


In [110]:
cv = CountVectorizer(tokenizer=custom_tokenizer, min_df=5)
cv.fit(X_train2)
X_train3 = cv.transform(X_train2)
X_test3 = cv.transform(X_test2)

In [116]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000, C=0.1)
lr.fit(X_train3, y_train2)
print(lr.score(X_train3, y_train2), lr.score(X_test3, y_test2))

0.9786 0.859
